# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

from lib.rag import RAG

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [20]:
import os
from enum import Enum
import json
from json import JSONDecodeError
import chromadb
from tavily import TavilyClient
from lib.vector_db import VectorStore
from lib.rag import RAG
from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.state_machine import Run
from lib.tooling import tool
from dotenv import load_dotenv

In [3]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
BASE_URL = "https://openai.vocareum.com/v1"

In [4]:

class OpenAIModel(str, Enum):
    GPT_41 = "gpt-4.1"  # Strong default choice for development tasks, particularly those requiring speed, responsiveness, and general-purpose reasoning.
    GPT_41_MINI = "gpt-4.1-mini"  # Fast and affordable, good for brainstorming, drafting, and tasks that don't require the full power of GPT-4.1.
    GPT_41_NANO = "gpt-4.1-nano"  # The fastest and cheapest model, suitable for lightweight tasks, high-frequency usage, and edge computing.
    GPT_35_TURBO = "gpt-3.5-turbo"

MODEL = OpenAIModel.GPT_41_MINI

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [84]:
chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay")
game_vector_store = VectorStore(collection)

@tool
def retrieve_game(query):
    """Tool Docstring:
   Semantic search: Finds most results in the vector DB
   args:
   - query: a question about game industry.

   You'll receive results as list. Each element contains:
   - ID: Unique ID of the document
   - Platform: like Game Boy, Playstation 5, Xbox 360...)
   - Name: Name of the Game
   - YearOfRelease: Year when that game was released for that platform
   - Description: Additional details about the game"""
    results = game_vector_store.query(query_texts=[query])
    documents = []
    for doc_id, content in zip(results['ids'][0], results['documents'][0]):
        documents.append(f"ID_{doc_id}: {content}")
    return documents

In [83]:
result = retrieve_game("Which one was the first 3D platformer Mario game?")
print(result)

["ID_009: [Nintendo 64] Super Mario 64 (1996) -genre: Platformer -publisher: Nintendo - A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.", 'ID_008: [Super Nintendo Entertainment System (SNES)] Super Mario World (1990) -genre: Platformer -publisher: Nintendo - A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land from Bowser.', 'ID_010: [GameCube] Super Smash Bros. Melee (2001) -genre: Fighting -publisher: Nintendo - A crossover fighting game featuring characters from various Nintendo franchises battling it out in dynamic arenas.']


In [79]:
results = game_vector_store.query("Who developed FIFA 21?")

In [81]:
print(results["ids"][0])

['015', '001', '011']


#### Evaluate Retrieval Tool

In [113]:
@tool
def evaluate_retrieval(query, retrieved_docs):
    """Tool Docstring:
    Based on the user's question and on the list of retrieved documents,
    it will analyze the usability of the documents to respond to that question.
    args:
    - question: original question from user
    - retrieved_docs: content of retrieved documents most similar to the user query in the Vector Database
    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """
    llm = LLM(model=MODEL, temperature=0)
    messages = [
        SystemMessage(content="You are an assistant for evaluating how relevant given document is to answer a question. Your task is to evaluate if the documents are enough to respond the query. Give a detailed explanation, so it's possible to take an action to accept it or not. The output format is a JSON object with the following keys: useful, description. Only reply with json string and no other text. Don't include ```json```"),
        UserMessage(
            content=(
                "Evaluate the following pieces of retrieved context to answer the question. "
                f"\n# Question: \n-> {query} "
                f"\n# Context: \n-> {retrieved_docs} "
                "\n# Answer: "
            )
        )
    ]
    ai_message = llm.invoke(messages)
    try:
        eval_result = json.loads(ai_message.content)
        eval_result["error"] = False
    except JSONDecodeError:
        print(f"Error: Invalid JSON returned by LLM: {ai_message.content}")
        eval_result = {"useful": False, "description": "Invalid JSON returned by LLM. Please try again.", "error": True}

    # return in string so that llm can process it directly
    return eval_result



In [114]:
eval = evaluate_retrieval("Which one was the first 3D platformer Mario game?", result)
print(eval)

{'useful': True, 'description': "The context includes information about 'Super Mario 64' released in 1996 on the Nintendo 64, described as a groundbreaking 3D platformer. It also mentions 'Super Mario World' from 1990 on the SNES, which is a classic platformer but not 3D, and 'Super Smash Bros. Melee' which is a fighting game. Since 'Super Mario 64' is explicitly identified as a 3D platformer and is the earliest 3D Mario platformer mentioned, the context is sufficient to answer that 'Super Mario 64' was the first 3D platformer Mario game.", 'error': False}


#### Game Web Search Tool

In [115]:
@tool
def game_web_search_tool(query):
    """
    Performs a web search for game-related information based on the provided query
    and returns the search results as structured data.

    This tool is specifically designed to fetch game-related details by
    querying web search platforms. It may be integrated into larger systems for
    fetching and processing game-related content dynamically.

    Args:
        query: A string representing the search query for games.

    Returns:
         urllib3.response.HTTPResponse: A response object containing the search results.
    """
    api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=api_key)
    result = client.search(query, search_depth="advanced", max_results=5, include_domains=["ign.com", "gamespot.com", "polygon.com", "eurogamer.net", "gamesradar.com"])
    return result

In [116]:
@tool
def game_web_search_tool(query):
    """
    Performs a web search for game-related information based on the provided query
    and returns the search results as structured data.

    This tool is specifically designed to fetch game-related details by
    querying web search platforms. It may be integrated into larger systems for
    fetching and processing game-related content dynamically.

    Args:
        query: A string representing the search query for games.

    Returns:
         List of dictionaries containing structured search result data, where each

    Returns:
        List of dictionaries containing structured search result data, where each 
        dictionary represents a single search result with related details such as 
        title, URL, and description.
    """
    if not query:
        raise ValueError("Search query cannot be empty")

    try:
        from tavily import TavilyClient
        api_key = os.getenv("TAVILY_API_KEY")
        client = TavilyClient(api_key=api_key)

        search_params = {
            'query': query + " video game",
            'search_depth': "advanced",
            'max_results': 5,
            'include_domains': [
                'ign.com',
                'gamespot.com',
                'polygon.com',
                'eurogamer.net',
                'gamesradar.com'
            ]
        }

        results = client.search(**search_params)
        return results

    except ImportError:
        raise RuntimeError("Tavily client not installed. Please install with: pip install tavily-python")
    except ConnectionError:
        raise ConnectionError("Failed to connect to search service")
    except Exception as e:
        raise RuntimeError(f"Unexpected error during search: {str(e)}")

In [ ]:
result = game_web_search_tool("Which one was the first 3D platformer Mario game?")
print(result)

### Agent

In [117]:
simple_agent = Agent(
    model_name=MODEL,
    instructions="""You are a helpful assistant for searching and answering questions about video games.
    You are given a tool to search the vector DB, a tool to evaluate the retrieved document, and a tool to search the web.
    ---
    Here are the guidelines for using the tools:
    1. Always search the vector DB first.
    2. Always evaluate the retrieved document using the evaluate tool.
    3. If the tool to search the vector DB returns no useful result, search the web. Provide citation if searched from web.
    ---
    Output Format:
    [Summary of the answer]
        short summary of the answer
    [Detailed explanation]
        detailed explanation of the answer
    [List of retrieved documents]
        [List of retrieved document ID]
            (ex: ID_001, ID_002, ID_003)
        [Evaluation result]
            whether the documents are enough to answer the question,
            (ex: useful, not useful, useful)
    [Citation]
        [Link to the relevant web page]
    [Additional information]
        [Error message]
        [Error code]
        [Error details]
        [Error traceback]
    """,
    tools=[retrieve_game, evaluate_retrieval, game_web_search_tool]
)

In [118]:

from typing import List
from lib.messages import BaseMessage
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")




In [119]:
simple_agent.memory.reset()
query1 = "When Pokémon Gold and Silver was released?"

run1 = simple_agent.invoke(query1)

messages1 = run1.get_final_state()["messages"]
print_messages(messages1)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
 -> (role = system, content = You are a helpful assistant for searching and answering questions about video games.
    You are given a tool to search the vector DB, a tool to evaluate the retrieved document, and a tool to search the web.
    ---
    Here are the guidelines for using the tools:
    1. Always search the vector DB first.
    2. Always evaluate the retrieved document using the evaluate tool.
    3. If the tool to search the vector DB returns no useful result, search the web. Provide citation if searched from web.
    ---
    Output Format:
    [Summary of the answer]
        short summary of the answer
    [Detailed explanation]
  

In [120]:
print(messages1[-1].content)

[Summary of the answer]
Pokémon Gold and Silver were released in 1999.

[Detailed explanation]
Pokémon Gold and Silver are second-generation Pokémon games that were released for the Game Boy Color platform. The release year for these games is 1999. These games introduced new regions, Pokémon, and gameplay mechanics, marking an important evolution in the Pokémon series.

[List of retrieved documents]
[ID_006]
[Evaluation result]
useful

[Citation]
None

[Additional information]
None


In [121]:
simple_agent.memory.reset()
query2 = "Which one was the first 3D platformer Mario game?"

run2 = simple_agent.invoke(query2)

messages2 = run2.get_final_state()["messages"]
print_messages(messages2)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
 -> (role = system, content = You are a helpful assistant for searching and answering questions about video games.
    You are given a tool to search the vector DB, a tool to evaluate the retrieved document, and a tool to search the web.
    ---
    Here are the guidelines for using the tools:
    1. Always search the vector DB first.
    2. Always evaluate the retrieved document using the evaluate tool.
    3. If the tool to search the vector DB returns no useful result, search the web. Provide citation if searched from web.
    ---
    Output Format:
    [Summary of the answer]
        short summary of the answer
    [Detailed explanation]
  

In [122]:
print(messages2[-1].content)

[Summary of the answer]
The first 3D platformer Mario game was Super Mario 64.

[Detailed explanation]
Super Mario 64, released in 1996 for the Nintendo 64, is recognized as the first 3D platformer game featuring Mario. It was groundbreaking at the time for its 3D gameplay and set new standards for the platformer genre. The game involves Mario's quest to rescue Princess Peach, marking a significant evolution from the earlier 2D platformers in the Mario series.

[List of retrieved documents]
ID_009
[Evaluation result]
useful

[Citation]
No web citation needed as the information is well-known and confirmed by the retrieved document.

[Additional information]
None


In [123]:
simple_agent.memory.reset()
query3 = "Was Mortal Kombat X realeased for Playstation 5?"

run3 = simple_agent.invoke(query3)

messages3 = run3.get_final_state()["messages"]
print_messages(messages3)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
 -> (role = system, content = You are a helpful assistant for searching and answering questions about video games.
    You are given a tool to search the vector DB, a tool to evaluate the retrieved document, and a tool to search the web.
    ---
    Here are the guidelines for using the tools:
    1. Always search the vector DB first.
    2. Always evaluate the retrieved document using the evaluate tool.
    3. If the tool to search the vector DB returns no useful result, search the web. Provide citation if searched from web.
    ---
    Output Format:
   

In [124]:
print(messages3[-1].content)

[Summary of the answer]
Mortal Kombat X was not released for PlayStation 5. It was originally released for PlayStation 4, Xbox One, and PC.

[Detailed explanation]
Based on the search results and available information, Mortal Kombat X was released for platforms including PlayStation 4, Xbox One, and PC. There is no indication or official release of Mortal Kombat X for the PlayStation 5 console. The game was launched before the PlayStation 5 existed, and no remastered or specific version for the PS5 has been documented.

[List of retrieved documents]
ID_005, ID_015, ID_004
Evaluation result: not useful

[Citation]
https://gamefaqs.gamespot.com/ps4/802908-mortal-kombat-x/data


In [125]:
simple_agent.memory.reset()
query4 = "Who developed FIFA 21"

run4 = simple_agent.invoke(query4)

messages4 = run4.get_final_state()["messages"]
print_messages(messages4)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
 -> (role = system, content = You are a helpful assistant for searching and answering questions about video games.
    You are given a tool to search the vector DB, a tool to evaluate the retrieved document, and a tool to search the web.
    ---
    Here are the guidelines for using the tools:
    1. Always search the vector DB first.
    2. Always evaluate the retrieved document using the evaluate tool.
    3. If the tool to search the vector DB returns no useful result, search the web. Provide citation if searched from web.
    ---
    Output Format:
   

In [126]:
print(messages4[-1].content)

[Summary of the answer]
FIFA 21 was developed by EA Sports.

[Detailed explanation]
From the web search results, it is known that FIFA 21 is a football simulation video game developed by EA Sports. EA Sports is a division of Electronic Arts that specializes in sports games, including the FIFA series. The search results mention features and updates related to FIFA 21 and include comments from members of the EA Sports team, confirming that EA Sports developed FIFA 21.

[List of retrieved documents]
["https://www.eurogamer.net/heres-our-first-look-at-fifa-21s-football-manager-style-career-mode", "https://www.gamesradar.com/fifa-21-debuts-first-next-gen-screenshots/", "https://www.eurogamer.net/fifa-21-review-high-scoring-fun-marred-by-pay-to-win-loot-boxes-again", "https://www.ign.com/articles/fifa-21-review", "https://www.ign.com/games/fifa-21-next-level-edition/gameplay"]
[Evaluation result]
useful

[Citation]
https://www.eurogamer.net/heres-our-first-look-at-fifa-21s-football-manager-s

In [127]:
simple_agent.memory.reset()

run5 = simple_agent.invoke("When was God of War Ragnarok released?")

messages5 = run5.get_final_state()["messages"]
print_messages(messages5)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
 -> (role = system, content = You are a helpful assistant for searching and answering questions about video games.
    You are given a tool to search the vector DB, a tool to evaluate the retrieved document, and a tool to search the web.
    ---
    Here are the guidelines for using the tools:
    1. Always search the vector DB first.
    2. Always evaluate the retrieved document using the evaluate tool.
    3. If the tool to search the vector DB returns no useful result, search the web. Provide citation if searched from web.
    ---
    Output Format:
    [Summary of the answer]
        short summary of the answer
    [Detailed explanation]
  

In [128]:
print(messages5[-1].content)

[Summary of the answer]
God of War Ragnarok was released on November 9, 2022.

[Detailed explanation]
God of War Ragnarok, the sequel to the 2018 God of War game, was officially released for PlayStation 4 and PlayStation 5 on November 9, 2022. This release date was confirmed by the developer Santa Monica Studio. Later, the game was also made available on PC, with the PC version releasing on September 19, 2024.

[List of retrieved documents]
[ID_004, ID_005, ID_012]
[Evaluation result]
not useful

[Citation]
https://www.polygon.com/23068988/god-of-war-ragnarok-release-date-ps5-ps4/
https://www.eurogamer.net/god-of-war-ragnarok-pc-release-date-time

[Additional information]
None


In [129]:
simple_agent.memory.reset()

run6 = simple_agent.invoke("What is Rockstar Games working on right now?")

messages6 = run6.get_final_state()["messages"]
print_messages(messages6)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
 -> (role = system, content = You are a helpful assistant for searching and answering questions about video games.
    You are given a tool to search the vector DB, a tool to evaluate the retrieved document, and a tool to search the web.
    ---
    Here are the guidelines for using the tools:
    1. Always search the vector DB first.
    2. Always evaluate the retrieved document using the evaluate tool.
    3. If the tool to search the vector DB returns no useful result, search the web. Provide citation if searched from web.
    ---
    Output Format:
   

In [130]:
print(messages6[-1].content)

[Summary of the answer]
Rockstar Games is currently working on Grand Theft Auto 6, which is officially announced with a release date set for 2025. Additionally, they are collaborating with Remedy Entertainment on remakes of Max Payne 1 and 2. They also plan to add classic Rockstar games like Bully and L.A. Noire to the GTA+ subscription service later in 2024.

[Detailed explanation]
Rockstar Games has confirmed two main projects in their current pipeline: Grand Theft Auto 6 and remakes of Max Payne 1 and 2. GTA 6 has been officially announced and is expected to be released in 2025. The first trailer revealed a new protagonist named Lucia. Remedy Entertainment is partnering with Rockstar to rebuild the two Max Payne games, although the release window for these remakes has not been announced yet.

Apart from these, Rockstar is enhancing its GTA+ subscription service by adding more classic games, including Bully and L.A. Noire, later in 2024. The company aims to enter a new era with sever